In [34]:
import requests
from bs4 import BeautifulSoup
import re
import threading
import time
import json
import pprint
from random import randint
import sqlite3

In [35]:

referers = ['tw.yahoo.com', 'www.google.com', 'http://www.msn.com/zh-tw/', 'http://www.pchome.com.tw/']
user_agents = ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
              'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
              'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)']

brand_list = ['AUDI', 'BENZ', 'BMW', 'FORD', 'HONDA', 'LEXUS', 'MAZDA', 'MITSUBISHI',
              'NISSAN', 'PORSCHE', 'SUZUKI', 'SUBARU', 'TOYOTA', 'VOLVO', 'VW']
def gen_headers():
    headers = {'user-agent': user_agents[randint(0, len(user_agents) - 1)],
               'referer': referers[randint(0, len(referers) - 1)]}
    return headers
headers=gen_headers()

In [33]:
with open('fail_line.txt','r') as f:
    data = f.readlines()
    
for line in data:
    carDic={}
    infos=line.split('\n')[0].split('|')
    carDic=json.loads(infos[2])
    url =infos[0]
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text,'lxml')
    carDic['color'] = soup.select('.mb-info span')[3].text.strip('色')
    carDic['gasoline'] = soup.select('.auto_standard span')[1].text.strip('車')
    carDic['wd'] = soup.select('.auto_standard span')[2].text
    carDic['doors'] = soup.select('.auto_standard span')[4].text
    
    certi_list=['sum','save','hot']
    if soup.select_one('.auto-check-t'):
        text =soup.select_one('.auto-check-t').text.lower()
        keyword = re.search(r'[a-z]+',text).group()
        if keyword in certi_list:
            carDic['certification'] = keyword
        else:
            carDic['certification'] = '其他'
    else:
        carDic['certification'] = 'na'

    equip=''
    for item in soup.select('.additionConfig'):
        equip += '{}|'.format(item.text)
    carDic['eqip'] = equip.strip('|')
    print(carDic)

{'source': '8891', 'title': '高雄巨將汽車宋小弟 車況超美超優 頂級天窗 雙安全氣囊 abs 認證+保固', 'brand': 'honda', 'model': 'cr-v', 'cc': '2.0l', 'transmission': '自排', 'mileage': '<b>17.9</b>萬公里', 'years': '2006', 'location': '高雄市', 'posttime': '2017-04-01 18:30:40', 'price': '30.8', 'color': '白', 'gasoline': '汽油', 'wd': '2WD', 'doors': '5門', 'certification': '其他', 'eqip': 'ABS防鎖死|中控鎖|防盜系統|倒車雷達|恆溫空調|多功能方向盤|真皮/皮革座椅'}
{'source': '8891', 'title': '高雄巨將汽車宋小弟 柴油動力 頂級四安 一手車原廠保養 認證+保固', 'brand': 'ford', 'model': 'focus', 'cc': '2.0l', 'transmission': '手自排', 'mileage': '<b>13.8</b>萬公里', 'years': '2011', 'location': '高雄市', 'posttime': '2017-04-01 18:57:20', 'price': '34.8', 'color': '銀', 'gasoline': '柴油', 'wd': '2WD', 'doors': '4門', 'certification': 'hot', 'eqip': 'ABS防鎖死|防盜系統|倒車雷達|恆溫空調|真皮/皮革座椅'}
{'source': '8891', 'title': '高雄巨將汽車宋小弟 一手車原廠保養 原廠保固中 恆溫空調 定速巡航', 'brand': 'toyota', 'model': 'camry', 'cc': '2.5l', 'transmission': '手自排', 'mileage': '<b>6.5</b>萬公里', 'years': '2014', 'location': '高雄市', 'posttime': '2017-04-02 17

In [31]:
def get_car_infos(headers,fileIn,fileOut):
    with open(fileIn,'r') as f:
        data = f.readlines()
    success=0
    for line in data:
        carDic={}
        infos=line.split('\n')[0].split('|')
        carDic=json.loads(infos[2])
        url =infos[0]

        count = 0
        
        while count<3:
            try:

                res = requests.get(url, headers=headers)
                soup = BeautifulSoup(res.text,'lxml')
                carDic['color'] = soup.select('.right-info .car-info ul li span')[7].text.strip('色')
                carDic['doors'] = soup.select('.car-detail-equipment .car-detail-base span')[1].text.split("：")[1]
                carDic['gasoline'] = soup.select('.car-detail-equipment .car-detail-base span')[2].text.split("：")[1].strip('車')
                carDic['wd'] = soup.select('.car-detail-equipment .car-detail-base span')[4].text.split("：")[1]
                
    #             print(soup.select('.check-show-info .tip')[1])
                if soup.select('div.tip > a'):
                    carDic['certificate'] = '其他'
                else:
                    carDic['certificate'] = 'NA'

                equip=''
                for item in soup.select('.car-equipment-show .info'):
                    equip += '{}|'.format(item.text)
                carDic['eqip'] = equip.strip('|')

                with open(fileOut,'a') as f:
                    f.write('{}\n'.format(json.dumps(carDic, ensure_ascii=False)))
                
                cursor.execute('INSERT INTO icars VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
               (carDic['source'], carDic['url'] , carDic['title'], carDic['brand'], carDic['model'], carDic['doors'], 
                carDic['color'], carDic['gasoline'], carDic['cc'], carDic['transmission'], carDic['equip'],
                carDic['mileage'], carDic['years'], carDic['location'], carDic['posttime'], 
                carDic['price'], carDic['certificate'], carDic['wd']))
                conn.commit()
                break
#             except IndexError:
                
            except Exception as e:
                count+=1
                if count==3:
                    message= 'fail url={},error:{}'.format(url, e)
                    print(message)
                    with open('fail_line0614.txt','a') as f:
                        f.write(line)
            finally:
                time.sleep(0.2)
    return success
headers=gen_headers()
fileIn='data0614-split.txt'
fileOut='detail0614-split.txt'
get_car_infos(headers,fileIn,fileOut)

In [36]:
conn = sqlite3.connect('/home/ubuntu/SQLiteDB/testCar.db')


In [39]:
cursor = conn.cursor()
cursor.execute('''CREATE TABLE cars
             (source text, url text, title text, brand text, model text, doors text, color text, gasoline text,
             cc INTEGER , transmission text, equip text, mileage INTEGER , years INTEGER , location text, 
             posttime INTEGER, price NUMERIC, certificate text, wd text)''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 63))



OperationalError: table cars already exists

In [166]:
with open('v2_result.txt','r') as f:
    data_r = f.readlines()
len(data_r)

44102

In [154]:
with open('v2_data.txt','r') as f:
    data_o = f.readlines()
data_o

['https://auto.8891.com.tw/usedauto-infos-1427554.html|1495770352|{"source": "8891", "url": "https://auto.8891.com.tw/usedauto-infos-1427554.html", "title": "【熊實在】★網路汽車買賣廣場★實車實圖實價登錄★汽車增貸周轉金★", "brand": "mitsubishi", "model": "lancer", "cc": 1600, "transmission": "手自排", "mileage": 160000, "years": 2005, "location": "臺中市", "posttime": 1495770352, "price": 5.8}\n',
 'https://auto.8891.com.tw/usedauto-infos-1434644.html|1496470235|{"source": "8891", "url": "https://auto.8891.com.tw/usedauto-infos-1434644.html", "title": "【主打0元購車專案】★零頭款★全貸★信用空白★八大行業★低月付 fb搜尋桃園小豬", "brand": "toyota", "model": "yaris", "cc": 1500, "transmission": "自排", "mileage": 20000, "years": 2015, "location": "桃園市", "posttime": 1496470235, "price": 25.0}\n',
 'https://auto.8891.com.tw/usedauto-infos-1434182.html|1496403973|{"source": "8891", "url": "https://auto.8891.com.tw/usedauto-infos-1434182.html", "title": "save認證 bmw 523i  m sport大包 可全貸", "brand": "bmw", "model": "5-series", "cc": 2500, "transmission": "手自排", "mile

In [152]:
with open('v2_result.txt','r') as f:
    data_r = f.readlines()
url_list=[]
for line in data_r:
    js=json.loads(line)
    url_list.append(js['url'])


In [138]:
with open('v2_fail_line.txt','r') as f:
    data = f.readlines()
len(data)

62

In [155]:
for line in data_o:
    if line.split('|')[0] not in url_list:
        with open('left_lines.txt','a') as f:
            f.write(line)

In [156]:
with open('left_lines.txt','r') as f:
    lines=f.readlines()
len(lines)

4961

In [164]:
len(data_o)

44108